In [1]:
import tensorflow as tf
print("Tensorflow version:", tf.__version__)
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)
try:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
except:
    # Invalid device or cannot modify virtual devices once initialized.
    pass

Tensorflow version: 2.9.1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


### Load a Dataset

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0 # 이미지 정규화하기

### Build a machine learning model

- 레이어를 쌓아 `tf.keras.Sequential` 모델 빌드

In [3]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(10)
])

- 모델은 각 이미지에 대해서 각 클래스에 대한 'logit' 또는 'log-odds'의 벡터를 반환

In [4]:
predictions = model(x_train[:1]).numpy()
predictions

array([[-0.9337505 , -0.19494703, -0.29569054,  0.08765808, -0.10929985,
        -0.3521268 ,  0.3264414 ,  0.51487756, -0.11004432,  0.529458  ]],
      dtype=float32)

- `tf.nn.softmax`는 위의 로짓을 각 클래스에 대한 확률로 변환

*'softmax' 함수를 네트워크의 마지막 레이어에 직접 입력하는 것이 가능하다. 이렇게 하면 모델 출력을 보다 직접적으로 해석할 수 있지만, 모든 모델에 대해 정확하고 수치적으로 안정적인 loss 계산을 제공하지 못하므로 권장하지 않는다.*

In [5]:
tf.nn.softmax(predictions).numpy()

array([[0.0381461 , 0.07985615, 0.07220313, 0.10593566, 0.08699704,
        0.06824111, 0.13450687, 0.1623982 , 0.08693229, 0.1647834 ]],
      dtype=float32)

- 학습을 위한 손실 함수 정의.
- `loss.SparseCategoricalCrossentropy`: 이 함수는 logit 벡터와 True 인덱스를 사용하고, 각 이미지에 대한 스칼라 loss를 반환

반환되는 loss는 실제 클래스의 음의 로그 확률과 같다. : 모델이 올바른 클래스를 확신하는 경우, loss는 0 이다.

*이 튜토리얼에서 다루는 mnist의 경우, 아직 학습되지 않은 모델은 무작위에 가까운 확률(각 클래스에 대해 1/10 (0~9까지 10개의 클래스))을 제공하므로 초기 loss는 `tf.math.log(1/10) ~= 2.3`에 가까워야 한다.*

In [6]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)


In [7]:
loss_fn(y_train[:1], predictions).numpy()

2.684708

- 학습을 시작하기 전에, Keras의 `Model.compile`을 사용하여 모델을 구성하고 컴파일 해야한다.
    - `optimizer`를 설정한다.
    - `loss function`을 설정한다.
    - `metric`을 설정하여 모델에 대해 평가할 기준을 지정한다. (여기서는 'accuracy':정확도)

In [8]:
model.compile(
    optimizer = 'adam',
    loss = loss_fn,
    metrics = ['accuracy']
)

### Train and evaluate model

- `model.fit`을 사용하여 model의 파라미터를 조정하고 loss를 최소화한다.

In [9]:
model.fit(x_train, y_train, epochs=10)

Epoch 1/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.2944 - accuracy: 0.9165
Epoch 2/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.1415 - accuracy: 0.9582
Epoch 3/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.1084 - accuracy: 0.9669
Epoch 4/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0888 - accuracy: 0.9729
Epoch 5/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0767 - accuracy: 0.9757
Epoch 6/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0659 - accuracy: 0.9791
Epoch 7/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0580 - accuracy: 0.9815
Epoch 8/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0527 - accuracy: 0.9829
Epoch 9/10
1875/1875 [==============================] - 9s 5ms/step - loss: 0.0478 - accuracy: 0.9842
Epoch 10/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.044

- `model.evaluate`을 사용하여 'validation-set' 또는 'test-set'을 통해 모델의 성능을 확인

In [10]:
model.evaluate(x_test, y_test, verbose=2)

313/313 - 1s - loss: 0.0732 - accuracy: 0.9784 - 1s/epoch - 4ms/step


[0.0732107013463974, 0.9783999919891357]

- 모델이 각 클래스에 대한 확률을 반화하도록 하려면 학습된 모델에 'softmax'를 추가하면 된다.

In [11]:
probability_model = tf.keras.Sequential([
    model,
    tf.keras.layers.Softmax()
])

In [12]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[2.77213665e-08, 2.70608687e-12, 8.38690966e-08, 5.65931305e-06,
        4.49088839e-14, 1.37167717e-07, 6.96044113e-18, 9.99990106e-01,
        1.15460042e-08, 4.04251568e-06],
       [1.01713427e-09, 6.75293677e-07, 9.99999285e-01, 2.04290274e-08,
        6.12091035e-20, 1.05369324e-10, 3.88459737e-10, 3.12349454e-17,
        5.34557953e-09, 1.97002899e-19],
       [2.85876145e-09, 9.99682426e-01, 1.76210233e-05, 2.65339452e-07,
        1.36495669e-06, 2.91723268e-08, 3.25693873e-06, 2.52197147e-04,
        4.29370375e-05, 1.36576768e-08],
       [9.99863267e-01, 3.85019843e-14, 1.11315174e-04, 1.29480576e-11,
        2.23080558e-08, 2.58178012e-08, 2.51346773e-05, 7.41386188e-08,
        7.95115571e-11, 8.85083580e-08],
       [1.75259322e-06, 7.01340353e-13, 6.13977818e-07, 2.86237894e-10,
        9.98783410e-01, 2.19724452e-08, 2.33533157e-07, 2.88961073e-05,
        1.62124479e-07, 1.18489226e-03]], dtype=float32)>